In [39]:
import pandas as pd
import numpy as np
from prediction.functions import aggregate_prediction_results

In [4]:
models = ['logit', 'ridge', 'lasso', 'enet', 'random_forest', 'lgb', 'nn3']
models2 = ['LOGIT', 'RIDGE', 'LASSO', 'ENET', 'RF', 'GB', 'NN3']
fs_methods = ['all', 'sfi', 'mdi', 'mda', 'granger', 'huang', 'IAMB', 'MMMB']
fs_methods2 = ['ALL', 'SFI', 'MDI', 'MDA', 'GRANGER', 'HUANG', 'IAMB', 'MMMB']
tickers = ['SPX Index', 'CCMP Index','RTY Index', 'SPX Basic Materials',
           'SPX Communications', 'SPX Consumer Cyclical',
           'SPX Consumer Non cyclical', 'SPX Energy', 'SPX Financial',
           'SPX Industrial', 'SPX Technology', 'SPX Utilities']
benchmark_name = 'return_direction'
metric = "auc"
oos_start_date = '2012-07-03'
oos_end_date = '2020-12-31'

In [5]:
oos_melt_predictions_df, oos_melt_benchmark_df, oos_melt_auc_df = aggregate_prediction_results(prediction_models=models,
                                                                                               fs_models=fs_methods,
                                                                                               evaluation_start_date=oos_start_date,
                                                                                               evaluation_end_date=oos_end_date,
                                                                                               ticker_names=tickers,
                                                                                               metric_name=metric,
                                                                                               tag='oos',
                                                                                               benchmark_name=benchmark_name)

/Users/dcuoliveira/Documents/Daniel/causal_feature/src/prediction/functions.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmark_df['model'] = benchmark_df['ticker']
/Users/dcuoliveira/Documents/Daniel/causal_feature/src/prediction/functions.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmark_df['fs'] = 'raw'


In [6]:
oos_melt_predictions_df.head()

,date,variable,value,model,fs,ticker
2139,2012-07-03,prediction,0.887274,LOGIT,ALL,SPX Index
2140,2012-07-05,prediction,0.000002,LOGIT,ALL,SPX Index
2141,2012-07-06,prediction,1.000000,LOGIT,ALL,SPX Index
2142,2012-07-09,prediction,0.000048,LOGIT,ALL,SPX Index
2143,2012-07-10,prediction,1.000000,LOGIT,ALL,SPX Index


In [8]:
model_oos_predictions_df = oos_melt_predictions_df.loc[(oos_melt_predictions_df['variable'] == 'prediction')&
                                                       (oos_melt_predictions_df['model'] == 'RF')&
                                                       (oos_melt_predictions_df['ticker'] == 'SPX Index')]
model_oos_predictions_df.head()

,date,variable,value,model,fs,ticker
2139,2012-07-03,prediction,0.526191,RF,ALL,SPX Index
2140,2012-07-05,prediction,0.605318,RF,ALL,SPX Index
2141,2012-07-06,prediction,0.590697,RF,ALL,SPX Index
2142,2012-07-09,prediction,0.596785,RF,ALL,SPX Index
2143,2012-07-10,prediction,0.545900,RF,ALL,SPX Index


In [21]:
melt_target_df = oos_melt_benchmark_df.loc[(oos_melt_benchmark_df['variable'] == 'return_direction')&
                                           (oos_melt_benchmark_df['model'] == 'SPX Index')&
                                           (oos_melt_benchmark_df['ticker'] == 'SPX Index')&
                                           (oos_melt_benchmark_df['fs'] == 'raw')]
melt_target_df.head()

,date,variable,value,model,fs,ticker
0,2012-07-03,return_direction,1.0,SPX Index,raw,SPX Index
1,2012-07-05,return_direction,0.0,SPX Index,raw,SPX Index
2,2012-07-06,return_direction,0.0,SPX Index,raw,SPX Index
3,2012-07-09,return_direction,0.0,SPX Index,raw,SPX Index
4,2012-07-10,return_direction,0.0,SPX Index,raw,SPX Index


## 1. Aggregating by forecasting model (across feature selection methods)

In [23]:
forecast_df = model_oos_predictions_df.pivot_table(index=['date'], columns=['model', 'fs'], values=['value'])
class_forecast_df = forecast_df.copy()
class_forecast_df[class_forecast_df <= 0.5] = 0
class_forecast_df[class_forecast_df > 0.5] = 1
class_forecast_df.head()

value                                       
model         RF                                       
fs           ALL GRANGER HUANG IAMB  MDA  MDI MMMB  SFI
date                                                   
2012-07-03   1.0     1.0   1.0  1.0  1.0  1.0  1.0  1.0
2012-07-05   1.0     1.0   1.0  0.0  1.0  1.0  1.0  1.0
2012-07-06   1.0     0.0   0.0  1.0  1.0  1.0  1.0  1.0
2012-07-09   1.0     1.0   1.0  1.0  1.0  1.0  1.0  1.0
2012-07-10   1.0     1.0   1.0  1.0  1.0  1.0  1.0  1.0

In [22]:
target_df = melt_target_df.pivot_table(index=['date'], columns=['model'], values=['value'])
target_df.head()

,value
model,SPX Index
date,
2012-07-03,1.0
2012-07-05,0.0
2012-07-06,0.0
2012-07-09,0.0
2012-07-10,0.0


In [34]:
class_forecast_df.columns.droplevel()[0][0] + '_' + class_forecast_df.columns.droplevel()[0][1] + '_' + 'resid'

'RF_ALL_resid'

In [36]:
resid_df = pd.concat([pd.DataFrame(target_df.values - class_forecast_df[[col]].values) for col in class_forecast_df.columns], axis=1)
resid_df.columns = [col[0][0] + '_' + col[0][1] + '_' + 'resid' for col in class_forecast_df.columns.droplevel()]
resid_df.head()

,R_F_resid,R_F_resid,R_F_resid,R_F_resid,R_F_resid,R_F_resid,R_F_resid,R_F_resid
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0
2,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [194]:

l_m = np.matrix(np.ones(shape=resid_df.shape[1]))

f_star_list = []
for i in range(forecast_df.shape[0]):
    e = np.matrix(l_m.T * target_df.values[i] - forecast_df.values[i])
    cov_e = e.T.dot(e)
    w_star = np.linalg.inv(l_m.dot(np.linalg.inv(cov_e)).dot(l_m.T)).item() * np.linalg.inv(cov_e).dot(l_m.T) 

    f_star = forecast_df.values[0].dot(w_star).item()
    f_star_list.append(f_star)

In [195]:
f_star_list

[0.9999999999999609,
 0.9620510210361868,
 0.694533283749859,
 0.7911353531550547,
 0.5200731291086576,
 0.5981210884791129,
 1.1346326255370691,
 0.6253944856271629,
 0.32235219503752566,
 0.7462190129668727,
 1.1204987357797953,
 1.014776484097585,
 0.6861620385640941,
 0.19901282692707234,
 0.027611467471391204,
 0.029889147009933836,
 2.307232672388821,
 0.37008315566443706,
 0.4978336842193143,
 0.7821234573474181,
 0.8449555972093536,
 0.2862699959763333,
 0.5650631800626651,
 0.6669895805177273,
 0.35579463695601365,
 0.3059697816955662,
 0.004137950060966844,
 0.3237706111968701,
 0.11493261794732179,
 -0.3629388699912184,
 0.6766648840492326,
 0.5681919184703681,
 0.7641270695038722,
 -0.11206284568914127,
 0.3843110218825734,
 0.6304443856798678,
 0.32490740416038566,
 0.12882879776051226,
 0.20501390701729516,
 0.27750678700317133,
 0.5407430932300634,
 0.3920019577803666,
 0.5595166317643074,
 0.6217593594536845,
 0.9147979310986851,
 0.18924969225541233,
 0.463024659393966